In [ ]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')
import torch

In [ ]:
ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'
MODEL_NAMES = ['../input/cotsmodels/yolo5m6_3000_2.pt',
               '../input/yolov5s6/f2_sub2.pt',
              '../input/reystarmodels/yolo5m6_3000_all_1.pt']
sizes=[4544,6400,3456]
flips=[False,False,False,False]
CONF      = 0.1
IOU       = 0.50
AUGMENT   = False
MIN_SZ    = 4

In [ ]:
# Train Data
df = pd.read_csv(f'{ROOT_DIR}/train.csv')
df['image_path'] = f'{ROOT_DIR}/train_images/video_'+df.video_id.astype(str)+'/'+df.video_frame.astype(str)+'.jpg'
df['annotations'] = df['annotations'].progress_apply(eval)
display(df.head(2))
df['num_bbox'] = df['annotations'].progress_apply(lambda x: len(x))
data = (df.num_bbox>0).value_counts()/len(df)*100
print(f"No BBox: {data[0]:0.2f}% | With BBox: {data[1]:0.2f}%")

In [ ]:
!mkdir -p /root/mylibs/
!cp -r '../input/yolov5detect/detection' /root/mylibs/
sys.path.insert(0,'/root/mylibs/')
!ls /root/mylibs/detection
from detection.detection_predict import get_img_predict_multi
from detection.yolov5.models.experimental import attempt_load

In [ ]:

def format_prediction_str(bboxes):
    annot = ''
    if len(bboxes)>0:
        for bb in bboxes:
            annot += f'{bb[4]:.2f} {int(bb[0])} {int(bb[1])} {int(bb[2])} {int(bb[3])}'
            annot +=' '
        annot = annot.strip(' ')
    return annot

def format_prediction(bboxes):
    pred_strings = []
    if len(bboxes)>0:
        for bb in bboxes:
            pred_strings.append(f'{bb[4]:.2f} {int(bb[0])} {int(bb[1])} {int(bb[2])} {int(bb[3])}')
    return ' '.join(pred_strings)

def check_result(bboxes,shape):
    if len(bboxes)==0:
        return []
    bboxes=np.array(bboxes)
    bboxes=bboxes[bboxes[:,4]>CONF]
    bboxes[bboxes<0]=0
    bboxes=bboxes[(bboxes[:,0]+bboxes[:,2]) < shape[1]]
    bboxes=bboxes[(bboxes[:,1]+bboxes[:,3]) < shape[0]]
    bboxes=bboxes[bboxes[:,2]>MIN_SZ]
    bboxes=bboxes[bboxes[:,3]>MIN_SZ]
    return bboxes

# Run code

In [ ]:
device='cuda:0'
   
models=[]
for model_name in MODEL_NAMES:
    detect_model = attempt_load(model_name, map_location=device)
    detect_model.eval()
    models.append(detect_model.half())
    # sizes.append(int(model_name.split('.')[0].split('_')[-1]))

In [ ]:
image_paths = df[df.num_bbox>1].sample(100).image_path.tolist()
for idx, path in enumerate(tqdm(image_paths)):
    img = cv2.imread(path)[...,::-1]
    bb_pred = get_img_predict_multi(models, img,sizes,flips)
    bb_pred = check_result(bb_pred,img.shape)
    annot   = format_prediction(bb_pred)
    print(idx,'---------',annot)
    if idx>5:
        break

# Run Test

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()  

In [ ]:
for idx, (img, pred_df) in tqdm(enumerate(iter_test)):
    #img=img[...,::-1]
    try:
        bb_pred = get_img_predict_multi(models,img, sizes, flips)
        bb_pred = check_result(bb_pred,img.shape)
        annot = format_prediction(bb_pred)
    except:
        annot = ''
    pred_df['annotations'] = annot
    env.predict(pred_df)
    if idx<3:
        print(annot)

In [ ]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()